# HEP Data

This noteboook reads in the outputs of combine to prepare tables of limits of the different points in the parameter space.

In [1]:
import numpy as np
import glob
import os
import uproot
import json
from tabulate import tabulate

In [2]:
# Define auxilary functions 

def get_limits(fn): # Returns quantile vs limits
    f = uproot.open(fn)
    limit = f["limit"]['limit'].array(library="np")
    quant = f["limit"]['quantileExpected'].array(library="np")
    if limit.shape[0] == 1:
        return -1
    else:
        return np.stack([quant,limit]) 
    
def get_SUEP_file(Mass=125, mdark=2, t=1, decay='generic', HT='1000', path="../"): # Returns filename
    if t < 10:
        tem = "{0:.2f}".format(t)
    else:
        tem = "{0:.1f}".format(t)
    tem = str(tem).replace(".","p")
    fname = os.path.join(
        "{}higgsCombineGluGluToSUEP_HT{}_T{}_mS{:.3f}_mPhi{:.3f}_T{:.3f}_mode{}_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root".format(path, HT, tem, Mass, mdark, t, decay)
    )
    if os.path.isfile(fname):
        return fname
    else:
        pass
     
def xs_scale(proc, era):
    xsec = 1.0
    with open(f"../config/xsections_{era}.json") as file:
            MC_xsecs = json.load(file)
    xsec  = MC_xsecs[proc]["xsec"]
    xsec *= MC_xsecs[proc]["kr"]
    xsec *= MC_xsecs[proc]["br"]
    #xsec *= 1000.0
    assert xsec > 0, "{} has a null cross section!".format(proc)
    return xsec

# Get all SUEP masses for a given mdark and T
def get_params_from_dir(path="../", era='2018', HT='1000'):
    """
    Grabs the parameters [T, mS, mPhi, mode, xsec] from each file from combine
    from a specific directory. The 'era' parameter is just used for the
    cross sections, which for SUEP don't change between years. HT parameter
    specifies the HT of the files.
    """

    # Get files
    combine_files = glob.glob("{}higgsCombineGluGluToSUEP_HT{}*.root".format(path, HT))

    # Filter out the parameters
    all_params = []
    for file in combine_files:
        file_trim = file.split("HT{}_".format(HT))[1]
        proc = 'GluGluToSUEP_HT{}_'.format(HT) + file_trim.split('.Asymptotic')[0]
        xsec = xs_scale(proc, era)
        t = float(file_trim.split("_")[0].replace("p",".")[1:])
        Mass = float(file_trim.split("_")[1][2:])
        mphi = float(file_trim.split("_")[2][4:])
        mode = file_trim.split('_')[4][4:]
        all_params.append([t,Mass,mphi,mode, xsec])

    return all_params


def create_limit_table(params, path, HT='1000'):
    """
    Expects params to be a list of lists of the shape: [t,Mass,mphi,mode, xsec].
    Returns table of limits.
    """

    rows = np.array(['T', 'mS', 'mPhi', 'Mode', 'Xsec', 'Obs', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma'])
    for param in params:
        lims = get_limits(get_SUEP_file(path=path, HT=HT, Mass=param[1], mdark=param[2], t=param[0], decay=param[3]))
        if lims.shape != (2,6):
            print(param, "has bad limits")
            continue

        lims = lims * param[-1]        

        _exp =lims[1][2]
        _s1p =lims[1][1] 
        _s1m =lims[1][3] 
        _s2p =lims[1][0] 
        _s2m =lims[1][4] 
        _obs =lims[1][5]   

        newline = [*param, _obs, _exp, _s2p, _s1p, _s1m, _s2m]
        rows = np.row_stack((rows, newline))
    
    return rows
    

In [3]:
# script parameters

ofile = '../ClosureSyst/hep_{}.txt'
itag = '../ClosureSyst/'
HT = '1000'

In [4]:
params = get_params_from_dir(itag, HT=HT)
table = create_limit_table(params, path=itag, HT=HT)

In [5]:
decays = ['generic', 'leptonic', 'hadronic']

# divide the table by decay mode, and save each separetely
for decay in decays:
    
    good_rows = table[:,3]==decay  # only select one decay mode
    good_rows[0] = True          # also save the labels
    table_decay = table[good_rows, :] 
    table_decay = table_decay[:, np.arange(table.shape[1]) != 3] # exclude the decay column from output table

    fname = ofile.format(decay)
    with open(fname, "w") as file:
        file.write(tabulate(table_decay[1:], headers=table_decay[0,:]))

    print("Written", fname)

Written ../ClosureSyst/hep_generic.txt
Written ../ClosureSyst/hep_leptonic.txt
Written ../ClosureSyst/hep_hadronic.txt
